# Scalability Part

This exp add the accuracy comparison of all algorithm on a sample of size 100 and 200.

In [2]:
import sys
sys.path.append("../../..")

import numpy as np
from sklearn.svm import SVC
from copy import deepcopy

from src.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize, generate_union_description, comp
)
import src

## 1. Generate Data

> Using [Adult](http://archive.ics.uci.edu/ml/datasets/Adult) Dataset.

Generate 100 size data:

| name | description |
| - | - |
| s_train_10p.csv | 10 points (5/5) |
| s_test_10p.csv | 100 points (50/50) |

Generate 100 size data:

| name | description |
| - | - |
| s_train_100p.csv | 100 points (50/50) |
| s_test_100p.csv | 100 points (50/50) |

Generate 200 size data:

| name | description |
| - | - |
| s_train_200p.csv | 200 points (100/100) |
| s_test_200p.csv | 100 points (50/50) |

In [ ]:
x_train_10, y_train_10, x_test_10, y_test_10, _ = preprocess_data('s_train_10p.csv', 's_test_10p.csv')
x_train_100, y_train_100, x_test_100, y_test_100, _ = preprocess_data('s_train_100p.csv', 's_test_100p.csv')
x_train_200, y_train_200, x_test_200, y_test_200, _ = preprocess_data('s_train_200p.csv', 's_test_200p.csv')

union_d_10 = generate_union_description(len(x_train_10), 10)
union_d_100 = generate_union_description(len(x_test_100), 10)
union_d_200 = generate_union_description(len(x_test_200), 10)

union_d_100_delete_last = deepcopy(union_d_100)
to_add_100 = 0
for i in len(union_d_100_delete_last): 
    if 99 in union_d_100_delete_last[i]:
        to_add_100 = i
        union_d_100_delete_last[i].remove(99)
        break 

union_d_10_delete_last = deepcopy(union_d_10)
to_add_10 = 0
for i in len(union_d_10_delete_last): 
    if 9 in union_d_10_delete_last[i]:
        to_add_10 = i
        union_d_10_delete_last[i].remove(9)
        break

## 2. Run algorithms

### 2.1 Add a single point

- pivot
- mc
- delta

In [ ]:
mc_plus_99 = src.mc_owen(x_train_100[:99], y_train_100[:99], x_test_100, y_test_100, SVC(decision_function_shape='ovo'), union_d_100_delete_last, m=100*100, proc_num=2)
save_npy('mc_plus_size99.npy', mc_plus_99)

In [ ]:
mc_plus_100 = src.mc_owen(x_train_100, y_train_100, x_test_100, y_test_100, SVC(decision_function_shape='ovo'), union_d_100, m=100*100, proc_num=3)
save_npy('mc_plus_size100.npy', mc_plus_100)

In [ ]:
mc_ov_100 = src.mc_owen(x_train_100, y_train_100, x_test_100, y_test_100, SVC(decision_function_shape='ovo'), union_d_100, m=100*30, proc_num=3)
save_npy('mc_size100m30.npy', mc_ov_100)

In [ ]:
size = 100
stone_m = 100

# 100 pivot
pivot_owen = src.PivotOwen(x_train_100[:99], y_train_100[:99], x_test_100, y_test_100, SVC(decision_function_shape='ovo'),
                                load_npy('mc_plus_size99.npy'), union_d_100_delete_last)

pivot_owen.prepare(m=99*stone_m)
save_npy('as_pivot_lsv_size' + str(size) + '.npy', pivot_owen.lov)

pivot_ov = pivot_owen.add_single_point(x_train_100[-1], y_train_100[-1], to_add_100, m=100*30, flags={'flag_lov': True})

save_npy('as_pivot_size' + str(size) +'m30' + '.npy', pivot_ov)

# 100 delta
delta_owen = src.DeltaOwen(x_train_100[:99], y_train_100[:99], x_test_100, y_test_100, SVC(decision_function_shape='ovo'),
                                load_npy('mc_plus_size99.npy'), union_d_100_delete_last)
delta_ov = delta_owen.add_single_point(x_train_100[-1], y_train_100[-1], to_add_100, m=100*30, flags={'flag_update': True})

save_npy('as_delta_size' + str(size) +'m30' + '.npy', delta_ov)

In [ ]:
size = 10
stone_m = 100

mc = src.mc_owen(x_train_10[:9], y_train_10[:9], x_test_10, y_test_10, SVC(decision_function_shape='ovo'), union_d_10_delete_last, m=stone_m*9)
save_npy('as_mc_plus_size9.npy', mc)

mc = src.mc_owen(x_train_10[:10], y_train_10[:10], x_test_10, y_test_10, SVC(decision_function_shape='ovo'), union_d_10, m=stone_m*10)
save_npy('as_mc_plus_size10.npy', mc)

mc = src.mc_owen(x_train_10[:10], y_train_10[:10], x_test_10, y_test_10, SVC(decision_function_shape='ovo'), union_d_10, m=30*10)
save_npy('as_mc_size10m30.npy', mc)

# 100 pivot
pivot_owen = src.PivotOwen(x_train_10[:9], y_train_10[:9], x_test_10, y_test_10, SVC(decision_function_shape='ovo'),
                              load_npy('as_mc_plus_size10.npy'), union_d_10_delete_last)
pivot_owen.prepare(m=9*stone_m)
save_npy('as_pivot_lsv_size' + str(size) + '.npy', pivot_owen.lov)

pivot_ov = pivot_owen.add_single_point(x_train_10[-1], y_train_10[-1], to_add_10, m=10*30, flags={'flag_lov': True})

save_npy('as_pivot_size' + str(size) +'m30' + '.npy', pivot_ov)

# 100 delta
delta_owen = src.DeltaOwen(x_train_10[:9], y_train_10[:9], x_test_10, y_test_10, SVC(decision_function_shape='ovo'),
                                load_npy('as_mc_plus_size9.npy'), union_d_10_delete_last)
delta_ov = delta_owen.add_single_point(x_train_10[-1], y_train_10[-1], to_add_10, m=10*30, flags={'flag_update': True})

save_npy('as_delta_size' + str(size) +'m30' + '.npy', delta_ov)

### 2.2 Add multiple points

- pivot
- mc
- delta

In [ ]:
size = 100
stone_m = 100

union_d_100_delete2 = deepcopy(union_d_100_delete_last)
to_add_100_2 = [to_add_100]
for i in len(union_d_100_delete2): 
    if 98 in union_d_100_delete2[i]:
        to_add_100_2.append(i)
        union_d_100_delete2[i].remove(98)
        break 

# 100 delta
delta_owen = src.DeltaOwen(x_train_100[:98], y_train_100[:98], x_test_100, y_test_100, SVC(decision_function_shape='ovo'),
                                load_npy('mc_plus_size98.npy'), union_d_100_delete2)
delta_owen.add_single_point(x_train_100[-2], y_train_100[-2], to_add_100_2[1], m=99*30, flags={'flag_update': True}, proc_num=2)
delta_ov = delta_owen.add_single_point(x_train_100[-1], y_train_100[-1], to_add_100, m=100*30, flags={'flag_update': True},proc_num=2)

save_npy('am_delta_size' + str(size) +'m30' + '.npy', delta_ov)

In [ ]:
size = 10
stone_m = 100

union_d_10_delete2 = deepcopy(union_d_10_delete_last)
to_add_10_2 = [to_add_10]
for i in len(union_d_10_delete2): 
    if 8 in union_d_10_delete2[i]:
        to_add_10_2.append(i)
        union_d_10_delete2[i].remove(8)
        break 

mc = src.mc_owen(x_train_10[:8], y_train_10[:8], x_test_10, y_test_10, SVC(decision_function_shape='ovo'), union_d_10_delete2, m=stone_m*8)
save_npy('am_mc_plus_size8.npy', mc)

mc = src.mc_owen(x_train_10[:10], y_train_10[:10], x_test_10, y_test_10, SVC(decision_function_shape='ovo'), union_d_10, m=stone_m*10)
save_npy('am_mc_plus_size10.npy', mc)

mc = src.mc_owen(x_train_10[:10], y_train_10[:10], x_test_10, y_test_10, SVC(decision_function_shape='ovo'), union_d_10, m=30*10)
save_npy('am_mc_size10m30.npy', mc)

# 100 pivot
pivot_owen = src.PivotOwen(x_train_10[:8], y_train_10[:8], x_test_10, y_test_10, SVC(decision_function_shape='ovo'), None, union_d_10_delete2)
pivot_owen.prepare(m=8*stone_m)
save_npy('am_pivot_lsv_size' + str(size) + '.npy', pivot_owen.lov)

pivot_owen.add_single_point(x_train_10[-2], y_train_10[-2], to_add_10_2[1], m=9*30, flags={'flag_lov': True})
pivot_owen.add_single_point(x_train_10[-1], y_train_10[-1], to_add_10, m=10*30, flags={'flag_lov': True})

save_npy('am_pivot_size' + str(size) +'m30' + '.npy', pivot_ov)

# 100 delta
delta_owen = src.DeltaOwen(x_train_10[:8], y_train_10[:8], x_test_10, y_test_10, SVC(decision_function_shape='ovo'),
                                load_npy('am_mc_plus_size8.npy'), union_d_10_delete2)
delta_owen.add_single_point(x_train_10[-2], y_train_10[-2], to_add_10_2[1], m=9*30, flags={'flag_update': True})
delta_ov = delta_owen.add_single_point(x_train_10[-1], y_train_10[-1], to_add_10, m=10*30, flags={'flag_update': True})
save_npy('am_delta_size' + str(size) +'m30' + '.npy', delta_ov)

### 2.3 Delete a single point

- mc
- delta

In [ ]:
# 200 delta
x_train_ = np.append(x_train_200, x_test_200[:1], axis=0)
y_train_ = np.append(y_train_200, y_test_200[:1])
x_test_ = x_test_200[1:, :]
y_test_ = y_test_200[1:]


delta_owen = src.DeltaOwen(x_train_, y_train_, x_test_, y_test_, SVC(decision_function_shape='ovo'),
                                load_npy('ds_mc_plus_size200.npy'))
delta_ov = delta_owen.del_single_point(len(y_train_) - 1, m=(len(y_train_)-1)*30, proc_num=25)
save_npy('ds_delta_size' + str(size) +'m30' + '.npy', delta_ov)


In [ ]:
size = 10
stone_m = 100

mc = src.mc_owen(np.append(x_train_10, x_train_100[-1:], axis=0), np.append(y_train_10, y_train_100[-1]), x_test_10, y_test_10, SVC(decision_function_shape='ovo'), m=100*11)
save_npy('ds_mc_plus_size11.npy', mc)

# 100 delta
delta_owen = src.DeltaShap(np.append(x_train_10, x_train_100[-1:], axis=0), np.append(y_train_10, y_train_100[-1]), x_test_10, y_test_10, SVC(decision_function_shape='ovo'),
                                load_npy('ds_mc_plus_size11.npy'))
delta_ov = delta_owen.del_single_point(10, m=10*30, flags={'flag_update': False})
save_npy('ds_delta_size' + str(size) +'m30' + '.npy', delta_ov)


### 2.4 Delete multiple points

- mc
- delta

In [ ]:
# 200 delta
x_train_ = np.append(x_train_200, x_test_200[:2], axis=0)
y_train_ = np.append(y_train_200, y_test_200[:2])
x_test_ = x_test_200[2:, :]
y_test_ = y_test_200[2:]


delta_owen = src.DeltaShap(x_train_, y_train_, x_test_, y_test_, SVC(decision_function_shape='ovo'),
                                load_npy('ds_mc_plus_size200.npy'))
delta_owen.del_single_point(len(y_train_) - 1, m=(len(y_train_)-1)*30, proc_num=25, flags={'flag_update': True})
delta_ov = delta_owen.del_single_point(len(y_train_) - 2, m=(len(y_train_)-2)*30, proc_num=25, flags={'flag_update': True})
save_npy('dm_delta_size' + str(size) +'m30' + '.npy', delta_ov)

In [ ]:
size = 10
stone_m = 100

mc = src.mc_owen(np.append(x_train_10, x_train_100[-2:], axis=0), np.append(y_train_10, y_train_100[-2:]), x_test_10, y_test_10, SVC(decision_function_shape='ovo'), m=100*12)
save_npy('dm_mc_plus_size12.npy', mc)

# 100 delta
delta_owen = src.DeltaShap(np.append(x_train_10, x_train_100[-2:], axis=0), np.append(y_train_10, y_train_100[-2:]), x_test_10, y_test_10, SVC(decision_function_shape='ovo'),
                                load_npy('dm_mc_plus_size12.npy'))
delta_owen.del_single_point(11, m=11*30, flags={'flag_update': True})
delta_owen.del_single_point(10, m=10*30, flags={'flag_update': True})
save_npy('dm_delta_size' + str(size) +'m30' + '.npy', delta_ov)

## 3. Comparison



In [ ]:
def compare():
    print('+----------------+')
    print('|   10 points    |')
    print('+----------------+')

    # add a single point 100 point
    print('>>> add a single point.')

    mc = load_npy('as_mc_size10m30.npy')
    mc_plus = load_npy('as_mc_plus_size10.npy')
    pivot = load_npy('as_pivot_size10m30.npy')
    delta = load_npy('as_delta_size10m30.npy')


    comp(mc_plus, mc, 'mc')
    comp(mc_plus, pivot, 'pivot')
    comp(mc_plus, delta, 'delta')

    print('>>> add multiple points.')

    mc = load_npy('am_mc_size10m30.npy')
    mc_plus = load_npy('am_mc_plus_size10.npy')
    pivot = load_npy('am_pivot_size10m30.npy')
    delta = load_npy('am_delta_size10m30.npy')# TODO

    comp(mc_plus, mc, 'mc')
    comp(mc_plus, pivot, 'pivot')
    comp(mc_plus, delta, 'delta')

    # del a single point 100 point
    print('>>> delete a single point.')
    delta = load_npy('ds_delta_size10m30.npy')

    comp(mc_plus, mc, 'mc')
    comp(mc_plus, delta, 'delta')

    print('>>> delete multiple points.')
    delta = load_npy('dm_delta_size10m30.npy')

    comp(mc_plus, mc, 'mc')
    comp(mc_plus, delta, 'delta')

    # =============================

    print('+-----------------+')
    print('|   100 points    |')
    print('+-----------------+')

    # add a single point 100 point
    print('>>> add a single point.')

    mc = load_npy('mc_size100m30.npy')
    mc_plus = load_npy('mc_plus_size100.npy')
    pivot = load_npy('as_pivot_size100m30.npy')
    delta = load_npy('as_delta_size100m30.npy')


    comp(mc_plus, mc, 'mc')
    comp(mc_plus, pivot, 'pivot')
    comp(mc_plus, delta, 'delta')

    # add multi points 100 point
    print('>>> add multiple points.')

    mc = load_npy('mc_size100m30.npy')
    mc_plus = load_npy('mc_plus_size100.npy')
    pivot = load_npy('am_pivot_size100m30.npy')
    delta = load_npy('am_delta_size100m30.npy')# TODO

    comp(mc_plus, mc, 'mc')
    comp(mc_plus, pivot, 'pivot')
    comp(mc_plus, delta, 'delta') # TODO

    # del a single point 100 point
    print('>>> delete a single point.')
    mc = load_npy('ds_mc_size100m30.npy')
    mc_plus = load_npy('ds_mc_plus_size100.npy')
    delta = load_npy('ds_delta_size100m30.npy')


    comp(mc_plus, mc, 'mc')
    comp(mc_plus, delta, 'delta')

    # del multi points 100 point
    print('>>> delete multiple points.')
    mc = load_npy('dm_mc_size100m30.npy')
    mc_plus = load_npy('dm_mc_plus_size100.npy')
    delta = load_npy('dm_delta_size100m30.npy')

    comp(mc_plus, mc, 'mc')
    comp(mc_plus, delta, 'delta')

    print('+-----------------+')
    print('|   200 points    |')
    print('+-----------------+')

    # add a single point 200 point
    print('>>> add a single point.')

    mc = load_npy('as_mc_size200m30.npy')
    mc_plus = load_npy('as_mc_plus_size200.npy')
    pivot = load_npy('as_pivot_size200m8.npy') # TODO
    delta = load_npy('as_delta_size200m8.npy') # TODO


    comp(mc_plus, mc, 'mc')
    comp(mc_plus, pivot, 'pivot')
    comp(mc_plus, delta, 'delta')

    # add multi points 200 point
    print('>>> add multiple points.')

    mc = load_npy('am_mc_size200m30.npy')
    mc_plus = load_npy('am_mc_plus_size200.npy')
    pivot = load_npy('am_pivot_size200m8.npy') # TODO
    delta = load_npy('am_delta_size200m15.npy') # TODO


    comp(mc_plus, mc, 'mc')
    comp(mc_plus, pivot, 'pivot')
    comp(mc_plus, delta, 'delta')

    # del a single point 200 point
    print('>>> delete a single point.')
    mc = load_npy('ds_mc_size200m30.npy')
    mc_plus = load_npy('ds_mc_plus_size200.npy')
    delta = load_npy('ds_delta_size200m30.npy')


    comp(mc_plus, mc, 'mc')
    comp(mc_plus, delta, 'delta')

    # del multi points 200 point
    print('>>> delete multiple points.')
    mc = load_npy('dm_mc_size200m30.npy')
    mc_plus = load_npy('dm_mc_plus_size200.npy')
    delta = load_npy('dm_delta_size200m30.npy')

    comp(mc_plus, mc, 'mc')
    comp(mc_plus, delta, 'delta')

In [ ]:
compare()

## 4 Additional exp part

> using data augmentation to make the train data size up tp 10,000

In [ ]:
def time_function(f, *args):
    """
    Call a function f with args and return the time (in seconds) that it took to execute.
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

In [ ]:
x_train, y_train, x_test, y_test, cols = preprocess_data('s_train_10000p.csv', 's_test_10000p.csv')
x_train = np.append(x_train, x_test[:2], axis=0)
y_train = np.append(y_train, y_test[:2], axis=0)
y_test = x_test[2:]
x_test = y_test[2:]

print('+-----------------------+')
print('| scalability time test |')
print('+-----------------------+')

print('>>> 10,000 points - add a single point')

# delta
delta_owen = src.DeltaShap(x_train[:9999], y_train[:9999], x_test, y_test,
                                SVC(decision_function_shape='ovo'), np.zeros(9999))
# time_function(delta_owen.add_single_point, x_train[-1], y_train[-1], 100)
delta_owen.add_single_point(x_train[-1], y_train[-1], m=100)

# pivot
pivot_owen = src.PivotShap(x_train[:9999], y_train[:9999], x_test, y_test,
                              SVC(decision_function_shape='ovo'), np.zeros(9999))
# time_function(delta_owen.add_single_point, x_train[-1], y_train[-1], 100)
pivot_owen.add_single_point(x_train[-1], y_train[-1], m=100)

# knn
knn_config = {
    'params': {
        'method': 'knn'
    },
}

knn_shap = src.HeurShap(x_train[:9999], y_train[:9999], x_test, y_test,
                              SVC(decision_function_shape='ovo'), np.zeros(9999), params=knn_config['params'])

knn_prepare_config = {
    'params': {
        'n_neighbors': 5,
    },
}
knn_shap.prepare(params=knn_prepare_config['params'])
# time_function(knn_shap.add_single_point, x_train[-1], y_train[-1])
knn_shap.add_single_point(x_train[-1], y_train[-1])

# knn+
knn_plus_config = {
    'params': {
        'method': 'knn+'
    },
}

knn_plus_shap = src.HeurShap(x_train[:9999], y_train[:9999], x_test, y_test,
                              SVC(decision_function_shape='ovo'), np.zeros(9999), params=knn_plus_config['params'])

knn_plus_prepare_config = {
    'flags': {
        'exact': False,
        'train': False
    },
    'params': {
        'n_neighbors': 5,
        'simi_type': 'ed',
        'f_shap': 'n*n',
        'rela': ['poly', 2],
        'train_idxs': [3, 11, 18],
        'train_svs': [np.zeros(9998)] * 3
    },
}
knn_plus_shap.prepare(knn_plus_prepare_config['flags'],
                      knn_plus_prepare_config['params'])
# time_function(knn_plus_shap.add_single_point, x_train[-1], y_train[-1])
knn_plus_shap.add_single_point(x_train[-1], y_train[-1])

In [ ]:
print('>>> 10,000 points - add multiple point')

# delta
delta_owen = src.DeltaShap(x_train[:9998], y_train[:9998], x_test, y_test,
                                SVC(decision_function_shape='ovo'), np.zeros(9998))
delta_owen.add_single_point(x_train[-2], y_train[-2], m=100, flags={'flag_update': True})
delta_owen.add_single_point(x_train[-1], y_train[-1], m=100, flags={'flag_update': True})

# pivot
pivot_owen = src.PivotShap(x_train[:9998], y_train[:9998], x_test, y_test,
                              SVC(decision_function_shape='ovo'), np.zeros(9998))
pivot_owen.add_single_point(x_train[-2], y_train[-2], m=100, flags={'flag_lov': True})
pivot_owen.add_single_point(x_train[-1], y_train[-1], m=100, flags={'flag_lov': True})

# knn
knn_config = {
    'params': {
        'method': 'knn'
    },
}

knn_shap = src.HeurShap(x_train[:9999], y_train[:9999], x_test, y_test,
                              SVC(decision_function_shape='ovo'), np.zeros(9999), params=knn_config['params'])

knn_prepare_config = {
    'params': {
        'n_neighbors': 5,
    },
}
knn_shap.prepare(params=knn_prepare_config['params'])
# time_function(knn_shap.add_multi_points, x_train[-2:], y_train[-2:])
knn_shap.add_multi_points(x_train[-2:], y_train[-2:])

# knn+
knn_plus_config = {
    'params': {
        'method': 'knn+'
    },
}

knn_plus_shap = src.HeurShap(x_train[:9998], y_train[:9998], x_test, y_test,
                                  SVC(decision_function_shape='ovo'), np.zeros(9998), params=knn_plus_config['params'])

knn_plus_prepare_config = {
    'flags': {
        'exact': False,
        'train': False
    },
    'params': {
        'n_neighbors': 5,
        'simi_type': 'ed',
        'f_shap': 'n*n',
        'rela': ['poly', 2],
        'train_idxs': [3, 11, 18],
        'train_svs': [np.zeros(9997)] * 3
    },
}
knn_plus_shap.prepare(knn_plus_prepare_config['flags'],
                      knn_plus_prepare_config['params'])
# time_function(knn_plus_shap.add_multi_points, x_train[-2:], y_train[-2:])
knn_plus_shap.add_multi_points(x_train[-2:], y_train[-2:])

In [ ]:
print('>>> 10,000 points - del a single point')

# yn-nn
del_num = 1
yn_shap = src.YnShap(x_train[:10001], y_train[:10001], x_test, y_test,
                          SVC(decision_function_shape='ovo'), np.zeros(10001))
yn_shap.del_num = del_num
shape = tuple([10001]) * (del_num + 2)
yn_shap.yn = np.zeros(shape)
yn_shap.nn = np.zeros(shape)
yn_shap.del_single_point(10000, flags={'exact': False})

# delta
delta_owen = src.DeltaShap(x_train[:10001], y_train[:10001], x_test, y_test,
                                SVC(decision_function_shape='ovo'), np.zeros(10001))
delta_owen.del_single_point(10000, m=100, flags={'flag_update': True})

# knn
knn_config = {
    'params': {
        'method': 'knn'
    },
}

knn_shap = src.HeurShap(x_train[:10001], y_train[:10001], x_test, y_test,
                              SVC(decision_function_shape='ovo'), np.zeros(10001), params=knn_config['params'])

knn_prepare_config = {
    'params': {
        'n_neighbors': 5,
    },
}
knn_shap.prepare(params=knn_prepare_config['params'])
knn_shap.del_single_point(10000)

# knn+
knn_plus_config = {
    'params': {
        'method': 'knn+'
    },
}

knn_plus_shap = src.HeurShap(x_train[:10001], y_train[:10001], x_test, y_test,
                                  SVC(decision_function_shape='ovo'), np.zeros(10001), params=knn_plus_config['params'])

knn_plus_prepare_config = {
    'flags': {
        'exact': False,
        'train': False
    },
    'params': {
        'n_neighbors': 5,
        'simi_type': 'ed',
        'f_shap': 'n*n',
        'rela': ['poly', 2],
        'train_idxs': [3, 11, 18],
        'train_svs': [np.zeros(10000)] * 3
    },
}
knn_plus_shap.prepare(knn_plus_prepare_config['flags'],
                      knn_plus_prepare_config['params'])
knn_plus_shap.add_multi_points([10000])

In [ ]:
print('>>> 10,000 points - del multiple point')

# ynn-nnn
del_num = 2
yn_shap = src.YnShap(x_train[:10002], y_train[:10002], x_test, y_test,
                          SVC(decision_function_shape='ovo'), np.zeros(10002))
yn_shap.del_num = del_num
shape = tuple([10001]) * (del_num + 2)
yn_shap.yn = np.zeros(shape)
yn_shap.nn = np.zeros(shape)
yn_shap.del_multi_points([10001, 10000], flags={'exact': False})

# delta
delta_owen = src.DeltaShap(x_train[:10002], y_train[:10002], x_test, y_test,
                                SVC(decision_function_shape='ovo'), np.zeros(10002))
delta_owen.del_single_point(10001, m=100, flags={'flag_update': True})
delta_owen.del_single_point(10000, m=100, flags={'flag_update': True})

# knn
knn_config = {
    'params': {
        'method': 'knn'
    },
}

knn_shap = src.HeurShap(x_train[:10002], y_train[:10002], x_test, y_test,
                              SVC(decision_function_shape='ovo'), np.zeros(10002), params=knn_config['params'])

knn_prepare_config = {
    'params': {
        'n_neighbors': 5,
    },
}
knn_shap.prepare(params=knn_prepare_config['params'])
knn_shap.del_single_point([10000, 10001])

# knn+
knn_plus_config = {
    'params': {
        'method': 'knn+'
    },
}

knn_plus_shap = src.HeurShap(x_train[:10002], y_train[:10002], x_test, y_test,
                                  SVC(decision_function_shape='ovo'), np.zeros(10002), params=knn_plus_config['params'])

knn_plus_prepare_config = {
    'flags': {
        'exact': False,
        'train': False
    },
    'params': {
        'n_neighbors': 5,
        'simi_type': 'ed',
        'f_shap': 'n*n',
        'rela': ['poly', 2],
        'train_idxs': [3, 11, 18],
        'train_svs': [np.zeros(10001)] * 3
    },
}
knn_plus_shap.prepare(knn_plus_prepare_config['flags'],
                      knn_plus_prepare_config['params'])
knn_plus_shap.add_multi_points([10000, 10001])